# ManuL

### Data preprocessing

Загрузка необходимых библиотек

In [26]:
import os.path

import pandas as pd
import numpy as np
import re
import urllib.parse
import requests
import tqdm
import pickle

Загрузка данных

In [4]:
df = pd.read_excel('../data/raw/UTexas Aptamer Database.xlsx')

Преобразуем колонки к snake_case

In [5]:
def to_snake_case(name):
    name = re.sub(r'[^\w\s]', ' ', name)
    name = re.sub(r'\s+', ' ', name)
    words = name.split()
    return '_'.join(word.lower() for word in words)

df.columns = [to_snake_case(col) for col in df.columns]

Просмотрим информацию о датасете

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1495 entries, 0 to 1494
Data columns (total 28 columns):
 #   Column                                                                                 Non-Null Count  Dtype  
---  ------                                                                                 --------------  -----  
 0   year_of_paper                                                                          1495 non-null   int64  
 1   link_to_pubmed_entry                                                                   1495 non-null   object 
 2   journals                                                                               1495 non-null   object 
 3   journal_doi                                                                            1495 non-null   object 
 4   citation                                                                               1495 non-null   object 
 5   type_of_nucleic_acid                                                        

Выведем 10 случайных строк

In [7]:
df.sample(10)

,year_of_paper,link_to_pubmed_entry,journals,journal_doi,citation,type_of_nucleic_acid,name_of_aptamer,target,aptamer_sequence,sequence_length,...,ph,molecular_weight_of_target,application_as_quoted_in_the_referenced_paper,post_selex_modifications_to_the_aptamer,additional_information,serial_number,parent_sequence_serial_number,corresponding_author_name_email_address,please_fill_out_the_form_for_any_feedbacks_comments,aptagen_cross_referencing_check_aptamer_chemistry_affinity_length_gc_content_sequence
922,2012,https://pubmed.ncbi.nlm.nih.gov/22222912/,Anal Bioanal Chem,https://doi.org/10.1007/s00216-011-5662-3,"Song KM, Jeong E, Jeon W, Cho M, Ban C. Aptase...",ssDNA,AMP18,Ampicillin,5'TTTAGTTGGGGTTCAGTTG3',19,...,8.0,Not reported,"Detection: "" A gold nanoparticle based dual fl...",Truncated: The ssDNA was shortened based on th...,Fam conjugated to 5' end of aptamer. The paper...,10000885,NaN,"Ban C, ciban@postech.ac.kr",https://forms.gle/n4TzuyddXQrHYJXF9,5'dTpdTpdTpdApdGpdTpdTpdGpdGpdGpdGpdTpdTpdCpdA...
1344,2019,https://pubmed.ncbi.nlm.nih.gov/30368278/,Talanta,https://doi.org/10.1016/j.talanta.2018.09.063,"Yu, F., Li, H., Sun, W., Zhao, Y., Xu, D., & H...",ssDNA,Ylac8,Lactoferrin,5'GCAGGACACCGTAACACGGGCTTTTGCTTTATCGTACCCTTTAT...,59,...,7.4,Not reported,"Detection: "" We report a novel method for effi...",Not applicable,NaN,10001302,NaN,"Li H, lihui@nju.edu.cn; Xu D, xudanke@nju.edu....",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
19,1993,https://pubmed.ncbi.nlm.nih.gov/7505429/,Nucleic Acids Res,https://doi.org/10.1093/nar/21.23.5509,"Giver, L., Bartel, D., Zapp, M., Pawul, A., Gr...",ssRNA,18,Rev protein of HIV-1 (minimal Rev-binding elem...,5'GCUCUUGGGCGCAGCCUCAAUGAGGCUGGAGGUACAAG3',38,...,8.0,Not reported,"Therapeutic: "" A population of improved Rev-bi...",Not applicable,"*Activity values are described as ""representin...",10000019,NaN,Ellington AD,https://forms.gle/n4TzuyddXQrHYJXF9,NaN
1006,2013,https://pubmed.ncbi.nlm.nih.gov/23968853/,Biomaterials,https://doi.org/10.1016/j.biomaterials.2013.07...,"Parekh, P., Kamble, S., Zhao, N., Zeng, Z., Po...",ssDNA,C2NP,CD30,5'ACTGGGCGAAACAAGTCTATTGACTATGGC3',30,...,7.4,Not reported,"Diagnostic and Therapeutic: "" CD30 is highly e...",Truncated version of c2 aptamer,NaN,10000965,NaN,"Zu, Y, yzu@tmhs.org",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
1445,2022,https://pubmed.ncbi.nlm.nih.gov/34383183/,Invest New Drugs,https://doi.org/10.1007/s10637-021-01161-y,"Tsogtbaatar, K., Sousa, D. A., Ferreira, D., T...",ssDNA,OS-7.9 aptamer,"Osteosarcoma MG-63 cell line, Human",5'ATCCAGAGTGACGCAGCAGGGCACATTGTTCACACACAGATCAC...,85,...,7.4,Not reported,"Diagnostic and Therapeutic: "" OS-7.9 could rec...",NaN,NaN,10001398,NaN,"Rodrigues, L; Email: lrmr@deb.uminho.pt",https://forms.gle/n4TzuyddXQrHYJXF9,5'dApdTpdCpdCpdApdGpdApdGpdTpdGpdApdCpdGpdCpdA...
1276,2018,https://pubmed.ncbi.nlm.nih.gov/30613143/,Int J Nanomedicine,https://doi.org/10.2147/IJN.S188003,"Chen, L., He, W., Jiang, H., Wu, L., Xiong, W....",ssDNA,PB aptamer,Prostate cancer bone metastasis (PB),5'CGCTCGATCGATCTAGATTCGCTCTATTGATGCCTGCGTGCGTG...,72,...,NaN,Not reported,"Drug Delivery and Therapeutic: "" PB is one of ...",Not applicable,NaN,10001234,NaN,"Qian, Y., qianyu@wibe.ac.cn",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
519,2004,https://pubmed.ncbi.nlm.nih.gov/15629041/,Genomics Proteomics Bioinformatics,https://doi.org/10.1016/s1672-0229(04)02005-4,"Yan, X., Gao, X., & Zhang, Z. (2004). Isolatio...",2'-amino-RNA,T3.11.7,"Tumor necrosis factor alpha (hTNFα), Human",5′GGGAGGACGAUGUUAGGAGUAUCUGAUGACAAUUCGGAGCUCCA...,59,...,7.4,Not reported,"Therapeutic and Diagnostic: ""Human tumor necro...",11 nucleotides of 5′- and 1 nucleotide from 3′...,"Modfied pool of 2'-NH2-RNA --> .2'-NH2 ,-modif...",10000495,NaN,"Zhang, Z., zhangzq@public3.bta.net.cn",https://forms.gle/n4TzuyddXQrHYJXF9,NaN
282,1999,https://pubmed.ncbi.nlm.nih.gov/10322029/,J Bacteriol,https://doi.org/10.1128/jb.181.10.3246-3255.1999,"Goodman, S. D., Velten, N. J., Gao, Q., R

### Работа с числовыми полями

In [8]:
df = df[df.pool_random_region != "TBD"]

Преобразуем молекулярную массу

In [9]:
def convert_to_da(value):
    if isinstance(value, (int, float)):
        return  value
    
    value = value.lower().strip()
    
    if pd.isna(value) or value in ["not reported", "n/a", "not reported"]:
        return np.nan
    
    value = str(value).replace(' ', '').strip()
    
    if 'kda' in value:
        return float(re.findall(r'\d+\.?\d*', value)[0]) * 1000
    elif 'da' in value or 'g/mol' in value:
        return float(re.findall(r'\d+\.?\d*', value)[0])
    else:
        return np.nan

In [10]:
df.molecular_weight_of_target = df.molecular_weight_of_target.apply(convert_to_da)

Преобразуем affinity

In [11]:
def affinity_value_convert(text):
    pattern = r'(\d+\.\d+|\d+)'
    
    range_match = re.match(r'Kd: (\d+\.\d+|\d+)-(\d+\.\d+|\d+) nM', text)
    if range_match:
        start_value = float(range_match.group(1))
        end_value = float(range_match.group(2))
        return (start_value + end_value) / 2
    else:
        match = re.search(pattern, text)
        if match:
            return float(match.group(0))
    return None

df.affinity = df.affinity.apply(affinity_value_convert)

Остальные поля просто преобразуем в числовой тип

In [12]:
num_columns = ["year_of_paper", "sequence_length", "gc_content", "serial_number", "ph", "affinity", "kd_nm", "pool_random_region", "molecular_weight_of_target"]

In [13]:
for col in num_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1494 entries, 0 to 1494
Data columns (total 28 columns):
 #   Column                                                                                 Non-Null Count  Dtype  
---  ------                                                                                 --------------  -----  
 0   year_of_paper                                                                          1494 non-null   int64  
 1   link_to_pubmed_entry                                                                   1494 non-null   object 
 2   journals                                                                               1494 non-null   object 
 3   journal_doi                                                                            1494 non-null   object 
 4   citation                                                                               1494 non-null   object 
 5   type_of_nucleic_acid                                                             

### Работа с категориальными столбцами

In [15]:
cat_columns = ['type_of_nucleic_acid', 'type_of_the_buffer',
               'divalent_salt']

In [16]:
df = pd.get_dummies(df, columns=cat_columns, prefix=cat_columns)

In [17]:
df.shape

(1494, 46)

### Представление SMILES and amino acid sequence 

In [18]:
def get_parameter_name(value):
    return urllib.parse.quote(value.replace('(', '').replace(')', '').replace(',', '').strip())

In [19]:
def get_smiles(compound_name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{get_parameter_name(compound_name)}/property/IsomericSMILES/JSON"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['PropertyTable']['Properties'][0]['IsomericSMILES']
    except:
        pass
    return None

In [20]:
def get_protein_sequence(protein_name):
    url = f"https://rest.uniprot.org/uniprotkb/search?query={get_parameter_name(protein_name)}&format=json&fields=sequence"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['results'][0]['sequence']['value']
    except:
        pass
    return None

In [21]:
def process_target(value):
    smiles = get_smiles(value)
    if smiles:
        return {'result': smiles, 'type': 'small_molecule'}
    
    sequence = get_protein_sequence(value)
    if sequence:
        return {'result': sequence, 'type': 'protein'}
    
    return {'type': 'unknown'}

In [22]:
if os.path.exists('../data/processed/target_info.pkl'):
    with open('../data/processed/target_info.pkl', 'rb') as f:
        target_info = pickle.load(f)
else:
    target_info = {}

In [23]:
unique_targets = df.target.unique()

In [24]:
if not os.path.exists('../data/processed/target_info.pkl'):
    for target in tqdm.tqdm(unique_targets):
        target_info[target] = process_target(target)

100%|██████████| 560/560 [18:40<00:00,  2.00s/it]


In [27]:
# if not os.path.exists('../data/processed/pre_3_step.pkl'):
#     with open('../data/processed/pre_3_step.pkl', 'wb') as f:
#         pickle.dump(df, f)

In [28]:
if not os.path.exists('../data/processed/target_info.pkl'):
    with open('../data/processed/target_info.pkl', 'wb') as f:
        pickle.dump(target_info, f)

In [29]:
df["target_type"] = df["target"].map(lambda x: target_info[x]['type'])

In [31]:
df["smiles"] = df["target"].map(lambda x: target_info[x]['result'] if target_info[x]['type'] == 'small_molecule' else None)

df["amino_acid_sequence"] = df["target"].map(lambda x: target_info[x]['result'] if target_info[x]['type'] == 'protein' else None)

In [36]:
df[["smiles", "amino_acid_sequence", "target_type"]].describe().T

,count,unique,top,freq
smiles,91,34,CC(=O)N[C@@H]1[C@H](C[C@@](O[C@H]1[C@@H]([C@@H...,18
amino_acid_sequence,785,234,MAPAMESPTLLCVALLFFAPDGVLAVPQKPKVSLNPPWNRIFKGEN...,36
target_type,1494,3,protein,785


In [37]:
df[["smiles", "amino_acid_sequence", "target_type"]]

,smiles,amino_acid_sequence,target_type
0,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,None,small_molecule
1,C1=CC=C2C(=C1)C(=O)C3=C(C2=O)C(=C(C=C3NC4=CC(=...,None,small_molecule
2,None,MKEFYISIETVGNNIVERYIDENGKERTREVEYLPTMFRHCKEESK...,protein
3,None,MKEFYISIETVGNNIVERYIDENGKERTREVEYLPTMFRHCKEESK...,protein
4,None,MIGGLFIYNHKGEVLISRVYRDDIGRNAVDAFRVNVIHARQQVRSP...,protein
...,...,...,...
1490,None,None,unknown
1491,None,MWVPVVFLTLSVTWIGAAPLILSRIVGGWECEKHSQPWQVLVASRG...,protein
1492,None,None,unknown
1493,None,None,unknown


К малым молекулам, имеющих `smiles` относятся `6,1%` строк dataset'а.
В свою очередь к белковым мишеням с `последовательностями аминокислот` относится `52.5%` записей. Тем самым для `41,4%` тип не был определён.

In [38]:
df.to_csv("../data/processed/processed.csv", index=False)